# To do
* split
* check dtypes
* convert to parquet
* start to play

# Machine info

In [1]:
!cat /proc/cpuinfo | grep -m1 "model name"
!awk '$3=="kB"{$2=$2/1024^2;$3="GB";} 1' /proc/meminfo | column -t | grep "MemTotal"

model name	: Intel(R) Core(TM) i3-4000M CPU @ 2.40GHz
MemTotal:           15.3896      GB


# Yellow Taxi Trip Data

* [2017](https://data.cityofnewyork.us/Transportation/2017-Yellow-Taxi-Trip-Data/biws-g3hs)
* [2018](https://data.cityofnewyork.us/Transportation/2018-Yellow-Taxi-Trip-Data/t29m-gskq)
* Full months by months [link](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page)

In [2]:
!ls -lh datasets/nyctaxi/raw/

total 20G
-rw-rw-r-- 1 baobob baobob 9.9G Apr  4  2019 2017_Yellow_Taxi_Trip_Data.csv
-rw-rw-r-- 1 baobob baobob 9.8G Feb 28 11:05 2018_Yellow_Taxi_Trip_Data.csv


In [1]:
import dask.dataframe as dd
import pandas as pd
from dask.distributed import Client, LocalCluster
client = Client()
client

Client Scheduler: tcp://127.0.0.1:40589 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 16.52 GB


In [2]:
df = dd.read_csv("datasets/nyctaxi/raw/*.csv")

In [3]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
npartitions=328,,,,,,,,,,,,,,,,,
,int64,object,object,int64,float64,int64,object,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [4]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,05/20/2017 05:41:25 PM,05/20/2017 05:58:43 PM,6,2.1,1,N,161,107,1,12.5,0.0,0.5,0.00,0.0,0.3,13.30
1,1,05/20/2017 05:41:26 PM,05/20/2017 05:51:26 PM,1,1.1,1,N,48,186,2,8.0,0.0,0.5,0.00,0.0,0.3,8.80
2,1,05/20/2017 05:41:26 PM,05/20/2017 06:00:03 PM,1,1.4,1,N,48,161,1,12.0,0.0,0.5,2.55,0.0,0.3,15.35
3,2,05/20/2017 05:41:26 PM,05/20/2017 05:53:11 PM,2,1.6,1,N,237,263,1,10.0,0.0,0.5,2.70,0.0,0.3,13.50
4,2,05/20/2017 05:41:26 PM,05/20/2017 05:46:17 PM,2,1.1,1,N,140,262,1,6.0,0.0,0.5,0.00,0.0,0.3,6.80


In [5]:
%%time
out = df.groupby(df.passenger_count).trip_distance.mean().compute()

CPU times: user 1min 33s, sys: 9.33 s, total: 1min 42s
Wall time: 14min 26s


In [6]:
out

passenger_count
0      2.762565
1      2.877378
2      3.091487
3      3.041540
4      3.111632
5      3.012087
6      2.990773
7      3.275516
8      5.036860
9      5.982670
192    1.010000
96     0.830000
Name: trip_distance, dtype: float64

In [7]:
out.reset_index(name="avg_distance")

,passenger_count,avg_distance
0,0,2.762565
1,1,2.877378
2,2,3.091487
3,3,3.041540
4,4,3.111632
5,5,3.012087
6,6,2.990773
7,7,3.275516
8,8,5.036860
9,9,5.982670


# Test 2018

In [1]:
import dask.dataframe as dd
import pandas as pd
from dask.distributed import Client, LocalCluster
client = Client()
client

Client Scheduler: tcp://127.0.0.1:34341 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 16.52 GB


In [2]:
df = dd.read_csv("datasets/nyctaxi/raw/2018_Yellow_Taxi_Trip_Data.csv",
                 parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])

In [2]:
11*20/60

3.6666666666666665